In [ ]:
!pip install --upgrade google-analytics-admin

In [ ]:
import os
import pandas
import datetime

from google.cloud import bigquery

from google.analytics.admin import AnalyticsAdminServiceClient
from google.analytics.admin_v1alpha.types import ListPropertiesRequest
from google.analytics.admin_v1alpha.types import ServiceLevel
from google.analytics.admin_v1alpha.types import AccessDateRange
from google.analytics.admin_v1alpha.types import AccessDimension
from google.analytics.admin_v1alpha.types import AccessMetric
from google.analytics.admin_v1alpha.types import RunAccessReportRequest

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/content/drive/MyDrive/Colab Notebooks/service-account.json"

account_id = 123456

start_date = datetime.datetime.strptime('2023-12-01', '%Y-%m-%d').date()
end_date = datetime.datetime.strptime('2023-12-31', '%Y-%m-%d').date()

datetime_delta = end_date - start_date

if __name__ == "__main__":
  client_analytics_admin = AnalyticsAdminServiceClient()
  client_bigquery = bigquery.Client()

  properties = client_analytics_admin.list_properties(
    ListPropertiesRequest(filter=f"parent:accounts/{account_id}", show_deleted=True)
  )

  for _day in range(datetime_delta.days + 1):
    date = start_date + datetime.timedelta(days = _day)

    _access_reports = []
    for property in properties:
      if(property.service_level != ServiceLevel.GOOGLE_ANALYTICS_360):
        continue

      access_reports = client_analytics_admin.run_access_report(
        RunAccessReportRequest(
          entity = property.name,
          dimensions = [
            AccessDimension(
              dimension_name = 'userEmail'
            ),
            AccessDimension(
              dimension_name = 'accessedPropertyId'
            ),
            AccessDimension(
              dimension_name = 'reportType'
            ),
            AccessDimension(
              dimension_name = 'revenueDataReturned'
            ),
            AccessDimension(
              dimension_name = 'costDataReturned'
            ),
            AccessDimension(
              dimension_name = 'userIP'
            ),
            AccessDimension(
              dimension_name = 'mostRecentAccessEpochTimeMicros'
            )
          ],
          metrics = [
            AccessMetric(
              metric_name = 'accessCount'
            )
          ],
          date_ranges = [
            AccessDateRange(
              start_date = date.strftime('%Y-%m-%d'),
              end_date = date.strftime('%Y-%m-%d')
            )
          ],
          return_entity_quota = True,
          expand_groups = True
        )
      )

      _access_report = []
      for rowIdx, row in enumerate(access_reports.rows):
        _dimensions_metrics = {}
        for i, dimension_value in enumerate(row.dimension_values):
          dimension_name = access_reports.dimension_headers[i].dimension_name
          if dimension_name.endswith("Micros"):
            dimension_value_formatted = datetime.datetime.utcfromtimestamp(int(dimension_value.value) / 1000000)
          else:
            dimension_value_formatted = dimension_value.value

          _dimensions_metrics[dimension_name] = dimension_value_formatted

        for i, metric_value in enumerate(row.metric_values):
          metric_name = access_reports.metric_headers[i].metric_name

          _dimensions_metrics[metric_name] = metric_value.value

        _access_report.append(_dimensions_metrics)

      _access_reports.extend(_access_report)

    dataframe = pandas.DataFrame(_access_reports)
    dataframe["accessCount"] = dataframe["accessCount"].astype(int)
    dataframe['date'] = date

    bigquery_dataset = client_bigquery.dataset('test_sinan')
    bigquery_table = bigquery_dataset.table('access_reports_' + date.strftime('%Y%m%d'))
    bigquery_job_config = bigquery.LoadJobConfig()
    bigquery_job_config.create_disposition = bigquery.CreateDisposition.CREATE_IF_NEEDED
    bigquery_job_config.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE

    bigquery_load_job = client_bigquery.load_table_from_dataframe(dataframe, bigquery_table, job_config = bigquery_job_config).result()
